In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 처리 및 분석을 위한 라이브러리 import
import pandas as pd  # 데이터프레임 생성 및 데이터 조작
import numpy as np  # 수학적 계산 및 배열 처리

In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/DP/bio_datas/"

In [ ]:
#파일 불러오기
dg_data = pd.read_csv(data_path + "DG-AssocMiner_miner-disease-gene.tsv", sep="\t")
protein_data = pd.read_csv(data_path + "train_data.csv")
gd_data = pd.read_csv(data_path + "curated_gene_disease_associations.txt", sep='\t')

<ipython-input-6-aa0183bad44d>:3: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  protein_data = pd.read_csv(data_path + "train_data.csv")


In [ ]:
print(protein_data.shape)
protein_data.head()

NameError: name 'protein_data' is not defined

In [ ]:
print(gd_data.shape)
gd_data.head()

In [ ]:
columns = [
    "UniProt_ID", "Entry_Name", "Gene ID", "RefSeq_Protein_ID", "GI_IDs",
    "PDB_IDs", "GO_Terms", "UniRef100", "UniRef90", "UniRef50",
    "UPI", "Additional_ID1", "Taxonomy", "Gene_Symbol", "Unused_1",
    "PubMed_IDs", "mRNA_IDs", "Protein_Seq_IDs", "Ensembl_Gene_ID",
    "Ensembl_Transcript_ID", "Ensembl_Protein_ID", "PubMed_More"
]
protein_data.columns = columns[:len(protein_data.columns)]

In [ ]:
protein_data.head()

In [ ]:
uniprot_df = gene_protein_id_mapping[['UniProt_ID', 'Gene ID', 'GO_Terms', 'PDB_IDs', 'PubMed_IDs', 'Ensembl_Protein_ID']]
uniprot_df.head()

In [ ]:
dg_data['Gene ID'] = dg_data['Gene ID'].astype(str)
uniprot_df['Gene ID'] = uniprot_df['Gene ID'].astype(str)

In [ ]:
# 병합 (Gene ID 기준으로)
merged_df = dg_data.merge(uniprot_df, on='Gene ID', how='left')

# 결과 확인
print(merged_df.shape)
merged_df.head()

# 관련 유전자, 단백질, 질병 데이터 파일 병합
질병 아이디와 이름, 유전자 아이디, 단백질 아이디를 기준으로 조인하여 한 데이터 파일로 만들어 준다. \
단백질 기능, 구조를 가지고 단백질과 질병과의 관계를 분석하고 질병을 예측하기 위해 전처리 과정으로 단계를 이어간다.

In [ ]:
# 1. explode
merged_df_exploded = merged_df.copy()
merged_df_exploded['Ensembl_Protein_ID'] = merged_df_exploded['Ensembl_Protein_ID'].fillna('')
merged_df_exploded = merged_df_exploded.assign(
    Ensembl_Protein_ID=merged_df_exploded['Ensembl_Protein_ID'].str.split(';')
).explode('Ensembl_Protein_ID')
merged_df_exploded['Ensembl_Protein_ID'] = merged_df_exploded['Ensembl_Protein_ID'].str.strip()

# 2. 버전 번호 제거 (.3 등 제거) → 정규식 활용
merged_df_exploded['Protein_ID_Clean'] = merged_df_exploded['Ensembl_Protein_ID'].str.replace(r'\.\d+$', '', regex=True)

# 3. protein_data에 맞추기 위해 앞에 "9606." 붙이기
merged_df_exploded['Protein_ID_Formatted'] = '9606.' + merged_df_exploded['Protein_ID_Clean']

# 4. # Disease ID에서 # 삭제
merged_df_exploded = merged_df_exploded.rename(columns={'# Disease ID': 'Disease ID'})

merged_df_exploded.head()

In [ ]:
# 1. 데이터 필요 컬럼만 추출
protein_data = protein_data[['protein1', 'protein2', 'combined_score']]
merged_df_exploded = merged_df_exploded[["Disease ID","Disease Name", "Gene ID",   "UniProt_ID",   "GO_Terms",   "PDB_IDs",   "PubMed_IDs", "Protein_ID_Formatted"]]

# 2. 병합용 리스트 생성
ppi_chunks = []

# 3. chunk 단위 병합 시작
chunk_size = 5000
for i in range(0, len(merged_df_exploded), chunk_size):
    chunk = merged_df_exploded.iloc[i:i+chunk_size].copy()

    # protein1 병합
    join1 = chunk.merge(protein_data, left_on='Protein_ID_Formatted', right_on='protein1', how='inner')

    # protein2 병합
    join2 = chunk.merge(protein_data, left_on='Protein_ID_Formatted', right_on='protein2', how='inner')

    # 두 병합 결과 저장
    ppi_chunks.append(pd.concat([join1, join2]))

# 4. 최종 병합 결과 결합
ppi_merged = pd.concat(ppi_chunks, ignore_index=True)


In [ ]:
ppi_merged.shape

NameError: name 'ppi_merged' is not defined

# 단백질 구조 데이터 파일 읽기

In [ ]:
#파일 불러오기
ps_data = pd.read_csv(data_path + "homo protein structure-results-csv.csv" , encoding='cp949')

In [ ]:
ps_data.shape

# ppi_merged파일과 ps_data파일 병합하기
uniprot_ID와 uniprotAccesstion 컬럼명을 기분으로 병합


In [ ]:
ppi_merged['UniProt_ID'] = ppi_merged['UniProt_ID'].str.strip()
ps_data['uniprotAccession'] = ps_data['uniprotAccession'].str.strip()

In [ ]:
# 컬럼 공백 제거 (안전장치)
ppi_merged.columns = [col.strip() for col in ppi_merged.columns]
ps_data.columns = [col.strip() for col in ps_data.columns]

# 병합 수행
merged_df = pd.merge(
    ppi_merged,
    ps_data,
    left_on='UniProt_ID',
    right_on='uniprotAccession',
    how='left'   # 기능 테이블 기준으로 유지
)

# 결과 확인
merged_df.head()

NameError: name 'ppi_merged' is not defined

In [ ]:
# common_ids = set(ppi_merged['UniProt_ID'].str.strip().str.upper()) & set(ps_data['uniprotAccession'].str.strip().str.upper())
# print("공통 UniProt ID 수:", len(common_ids))
merged_df.isnull().sum()

# 미지의 단백질 구조 데이터 파일 읽기:
단백질 파일이 fasta형태로 되어있어서 dataframe으로 바꿔준 다음 저장하였다.

In [ ]:
!pip install biopython

In [ ]:
import pandas as pd
from Bio import SeqIO

fasta_path = data_path + "uniprotkb_uncharacterized_protein_AND_r_2025_03_21.fasta (1)"

data = []
for record in SeqIO.parse(fasta_path, "fasta"):
    data.append({
        "description": record.description,
        "sequence": str(record.seq)
    })

df = pd.DataFrame(data)

In [ ]:
import re

# description 파싱
def parse_description(desc):
    pattern = (
        r'^sp\|(?P<uniprotAccession>\S+)\|(?P<entry_name>\S+)\s+'
        r'(?P<protein_name>.*?)\s+OS=(?P<organism>.*?)\s+'
        r'OX=(?P<tax_id>\d+)\s+GN=(?P<gene>\S+)\s+'
        r'PE=(?P<protein_existence>\d+)\s+SV=(?P<sequence_version>\d+)'
    )
    match = re.match(pattern, desc)
    if match:
        return pd.Series(match.groupdict())
    else:
        return pd.Series([None]*8, index=[
            'uniprotAccession', 'entry_name', 'protein_name', 'organism',
            'tax_id', 'gene', 'protein_existence', 'sequence_version'
        ])

# 파싱 적용
parsed = df['description'].apply(parse_description)

# 병합
final_df = pd.concat([parsed, df['sequence']], axis=1)

# 중복 컬럼 제거 및 정리
print(final_df.head())

  uniprotAccession   entry_name                            protein_name  \
0           O14744   ANM5_HUMAN  Protein arginine N-methyltransferase 5   
1           O14978  ZN263_HUMAN                 Zinc finger protein 263   
2           O15194  CTDSL_HUMAN      CTD small phosphatase-like protein   
3           O43633  CHM2A_HUMAN  Charged multivesicular body protein 2a   
4           O75884  RBBP9_HUMAN                  Serine hydrolase RBBP9   

       organism tax_id    gene protein_existence sequence_version  \
0  Homo sapiens   9606   PRMT5                 1                4   
1  Homo sapiens   9606  ZNF263                 1                2   
2  Homo sapiens   9606  CTDSPL                 1                2   
3  Homo sapiens   9606  CHMP2A                 1                1   
4  Homo sapiens   9606   RBBP9                 1                2   

                                            sequence  
0  MAAMAVGGAGGSRVSSGRDLNCVPEIADTLGAVAKQGFDFLCMPVF...  
1  MASGPGSQEREGLLIVKLEED

In [ ]:
final_df.head()

,uniprotAccession,entry_name,protein_name,organism,tax_id,gene,protein_existence,sequence_version,sequence
0,O14744,ANM5_HUMAN,Protein arginine N-methyltransferase 5,Homo sapiens,9606,PRMT5,1,4,MAAMAVGGAGGSRVSSGRDLNCVPEIADTLGAVAKQGFDFLCMPVF...
1,O14978,ZN263_HUMAN,Zinc finger protein 263,Homo sapiens,9606,ZNF263,1,2,MASGPGSQEREGLLIVKLEEDCAWSQELPPPDPGPSPEASHLRFRR...
2,O15194,CTDSL_HUMAN,CTD small phosphatase-like protein,Homo sapiens,9606,CTDSPL,1,2,MDGPAIITQVTNPKEDEGRLPGAGEKASQCNVSLKKQRSRSILSSF...
3,O43633,CHM2A_HUMAN,Charged multivesicular body protein 2a,Homo sapiens,9606,CHMP2A,1,1,MDLLFGRRKTPEELLRQNQRALNRAMRELDRERQKLETQEKKIIAD...
4,O75884,RBBP9_HUMAN,Serine hydrolase RBBP9,Homo sapiens,9606,RBBP9,1,2,MASPSKAVIVPGNGGGDVTTHGWYGWVKKELEKIPGFQCLAKNMPD...


# 미지 단백질 구조 파일 매핑 및 병합하기
uniprotId랑 uniprotAccession이랑 병합

In [ ]:
merged_df['UniProt_ID'] = merged_df['UniProt_ID'].str.strip()
final_df['uniprotAccession'] = final_df['uniprotAccession'].str.strip()
final_df = final_df.rename(columns={'uniprotAccession': 'uniprotAccession_unchar'})

NameError: name 'merged_df' is not defined

In [ ]:
# 컬럼 공백 제거 (안전장치)
merged_df.columns = [col.strip() for col in merged_df.columns]
final_df.columns = [col.strip() for col in final_df.columns]

# 병합 수행
merged_df = pd.merge(
    merged_df,
    final_df,
    left_on='UniProt_ID',
    right_on='uniprotAccession_unchar',
    how='left'   # 기능 테이블 기준으로 유지
)

# 결과 확인
merged_df.head()

In [ ]:
print("병합된 단백질 수:", merged_df['uniprotAccession_unchar'].notnull().sum())
print("병합 실패 (NaN) 수:", merged_df['uniprotAccession_unchar'].isnull().sum())

In [ ]:
# 병합이 성공한 행만 필터링 (즉, uniprotAccession_unchar가 NaN이 아닌 행)
merged_success = merged_df[merged_df['uniprotAccession_unchar'].notnull()]

In [ ]:
merged_success.to_csv("/content/drive/MyDrive/Colab Notebooks/DP/bio_datas/merged_success.csv", index=False, encoding='utf-8-sig')

In [ ]:
print("전체 행 수:", len(merged_success))
print("NaN 수:", merged_success['uniprotAccession_unchar'].isnull().sum())

# 데이터 전처리
데이터 파일이 너무 커서 샘플 1% 저장하여 그 부분만 테스트로 진행할 예정

In [ ]:
import pandas as pd

# 1. 1% 샘플링 (무작위 추출, seed 고정하면 재현 가능)
sample_ppi = merged_success.sample(frac=0.01, random_state=42)

# 2. 저장 경로 지정
data_path = "/content/drive/MyDrive/Colab Notebooks/DP/bio_datas/"
output_file = data_path + 'merged_success_sample1.csv'  # 파일명 수정

# 3. CSV 저장
sample_ppi.to_csv(output_file, index=False)

print(f'✅ 샘플 1% 저장 완료: {output_file}')

In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/DP/bio_datas/"
train_data = pd.read_csv(data_path + "merged_success_sample1.csv", sep=',')

# 확인
print(train_data.shape)
train_data.head()

In [ ]:
# 1️⃣ 유전자 이름 컬럼 통일 ('gene_y' → 'geneSymbol')
if 'gene_y' in protein_data.columns:
    protein_data.rename(columns={'gene_y': 'geneSymbol'}, inplace=True)

# 2️⃣ gd_data 필터링 (신뢰도 기준)
gd_filtered = gd_data[
    (gd_data["score"] >= 0.3) &
    (gd_data["NumberOfPubmeds"] >= 3)
].copy()

# 질병당 유전자 수 ≥ 3개 필터링
valid_diseases = gd_filtered['diseaseId'].value_counts()
valid_diseases = valid_diseases[valid_diseases >= 3].index
gd_filtered = gd_filtered[gd_filtered['diseaseId'].isin(valid_diseases)]

# 3️⃣ 병합: geneSymbol 기준
merged = pd.merge(protein_data, gd_filtered, on='geneSymbol', how='inner')

# 4️⃣ 결과 확인
print(f"병합된 데이터 수: {merged.shape[0]}")
display(merged[['geneSymbol', 'diseaseName', 'score', 'UniProt_ID', 'sequence']].head())

# 2. 저장 경로 지정
data_path = "/content/drive/MyDrive/Colab Notebooks/DP/bio_datas/"
output_file = data_path + 'merged.csv'  # 파일명 수정

# 3. CSV 저장
merged.to_csv(output_file, index=False)


병합된 데이터 수: 20702


,geneSymbol,diseaseName,score,UniProt_ID,sequence
0,SCN5A,Long QT Syndrome,0.499446,Q14524,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
1,SCN5A,Long QT Syndrome,0.499446,Q14524,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
2,SCN5A,Long QT Syndrome,0.499446,Q14524,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
3,SCN5A,Long QT Syndrome,0.499446,Q14524,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
4,SCN5A,Long QT Syndrome,0.499446,Q14524,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...


In [ ]:
merged.head()


,index,Disease ID,Disease Name,Gene ID,UniProt_ID,GO_Terms,PDB_IDs,PubMed_IDs,protein1,protein2,...,sequence_version,sequence,geneId,geneName,diseaseId,diseaseName,score,NumberOfPubmeds,associationType,source
0,9732,C1142166,Brugada Syndrome (disorder),6331,Q14524,GO:0005901; GO:0009986; GO:0005783; GO:0014704...,2KBI:A; 2L53:B; 4DCK:A; 4DJC:B; 4JQ0:D; 4OVN:F...,1309946; 12358675; 14500339; 12454206; 1611520...,ENSP00000328968,ENSP00000261007,...,2,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,6331,"sodium channel, voltage gated, type V alpha su...",umls:C0023976,Long QT Syndrome,0.499446,135,"Biomarker, GeneticVariation","BeFree, CLINVAR, CTD_human, GAD, LHGDN"
1,9733,C1142166,Brugada Syndrome (disorder),6331,Q14524,GO:0005901; GO:0009986; GO:0005783; GO:0014704...,2KBI:A; 2L53:B; 4DCK:A; 4DJC:B; 4JQ0:D; 4OVN:F...,1309946; 12358675; 14500339; 12454206; 1611520...,ENSP00000328968,ENSP00000302486,...,2,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,6331,"sodium channel, voltage gated, type V alpha su...",umls:C0023976,Long QT Syndrome,0.499446,135,"Biomarker, GeneticVariation","BeFree, CLINVAR, CTD_human, GAD, LHGDN"
2,9734,C1142166,Brugada Syndrome (disorder),6331,Q14524,GO:0005901; GO:0009986; GO:0005783; GO:0014704...,2KBI:A; 2L53:B; 4DCK:A; 4DJC:B; 4JQ0:D; 4OVN:F...,1309946; 12358675; 14500339; 12454206; 1611520...,ENSP00000328968,ENSP00000311005,...,2,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,6331,"sodium channel, voltage gated, type V alpha su...",umls:C0023976,Long QT Syndrome,0.499446,135,"Biomarker, GeneticVariation","BeFree, CLINVAR, CTD_human, GAD, LHGDN"
3,9735,C1142166,Brugada Syndrome (disorder),6331,Q14524,GO:0005901; GO:0009986; GO:0005783; GO:0014704...,2KBI:A; 2L53:B; 4DCK:A; 4DJC:B; 4JQ0:D; 4OVN:F...,1309946; 12358675; 14500339; 12454206; 1611520...,ENSP00000328968,ENSP00000265362,...,2,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,6331,"sodium channel, voltage gated, type V alpha su...",umls:C0023976,Long QT Syndrome,0.499446,135,"Biomarker, GeneticVariation","BeFree, CLINVAR, CTD_human, GAD, LHGDN"
4,9736,C1142166,Brugada Syndrome (disorder),6331,Q14524,GO:0005901; GO:0009986; GO:0005783; GO:0014704...,2KBI:A; 2L53:B; 4DCK:A; 4DJC:B; 4JQ0:D; 4OVN:F...,1309946; 12358675; 14500339; 12454206; 1611520...,ENSP00000328968,ENSP00000431822,...,2,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,6331,"sodium channel, voltage gated, type V alpha su...",umls:C0023976,Long QT Syndrome,0.499446,135,"Biomarker, GeneticVariation","BeFree, CLINVAR, CTD_human, GAD, LHGDN"


# 딥러닝 학습
위 병합한 데이터 파일을 가지고 딥러닝 학습을 시작한다. \
미지의 단백질이 어떤 단백질하고 관련이 있고, 어느 질병을 초래할 것인지 예측하는 코드를 구현한다. \
GCN과 GAT 모델을 통해 둘을 비교하여 성능을 평가한다.

# 여기부터 시작하면됩니다.

In [ ]:
# ✅ PyTorch + PyG 설치 (CPU 버전)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-geometric

Looking in indexes: https://download.pytorch.org/whl/cpu
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html


# 아래 코드는 피처로 단백질-단백질 관계, 단백질 구조, 유전자-질병 관계를 피처로 재생성 하기 위함.

In [ ]:
import pandas as pd
import numpy as np
import torch
import networkx as nx
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 1️⃣ 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DP/bio_datas/train_data.csv")

# 1. 노드 ID 추출 및 인덱싱
all_proteins = pd.unique(df[['protein1', 'protein2']].values.ravel())
protein_to_idx = {protein: idx for idx, protein in enumerate(all_proteins)}

# 2. edge_index 생성
edges = df[['protein1', 'protein2']].dropna().values
edge_index = torch.tensor([[protein_to_idx[a], protein_to_idx[b]] for a, b in edges], dtype=torch.long).T

# 3. 자동 피처 생성
df_feat = df.copy()

# 2️⃣ 결측값 처리
df_feat['sequence'] = df_feat['sequence'].fillna('')
df_feat['GO_Terms'] = df_feat['GO_Terms'].fillna('')
df_feat['PubMed_IDs'] = df_feat['PubMed_IDs'].fillna('')
df_feat['globalMetricValue'] = df_feat['globalMetricValue'].fillna(0)
df_feat['organismScientificName'] = df_feat.get('organismScientificName', pd.Series(['Unknown'] * len(df_feat))).fillna('Unknown')

# 수치형 피처 생성
df_feat['sequence_length'] = df_feat['sequence'].fillna('').apply(len)
df_feat['go_term_count'] = df_feat['GO_Terms'].fillna('').apply(lambda x: len(x.split(';')) if x else 0)
df_feat['pubmed_count'] = df_feat['PubMed_IDs'].fillna('').apply(lambda x: len(str(x).split(';')) if pd.notnull(x) else 0)

# 그래프 기반 피처
ppi_edges = df[['protein1', 'protein2', 'combined_score']].dropna()
ppi_graph = nx.from_pandas_edgelist(ppi_edges, source='protein1', target='protein2', edge_attr='combined_score')

ppi_degree = {prot: val for prot, val in ppi_graph.degree()}
ppi_avg_score = {node: np.mean([d['combined_score'] for _, _, d in ppi_graph.edges(node, data=True)]) for node in ppi_graph.nodes()}

df_feat['ppi_degree'] = df_feat['protein1'].map(ppi_degree).fillna(0)
df_feat['ppi_avg_score'] = df_feat['protein1'].map(ppi_avg_score).fillna(0)

df_feat['pdb_count'] = df_feat['PDB_IDs'].fillna('').apply(lambda x: len(x.split(';')) if x else 0)

gene_disease_count = df_feat.groupby('Gene ID')['Disease ID'].nunique()
df_feat['gene_disease_count'] = df_feat['Gene ID'].map(gene_disease_count).fillna(0)

# 수치형 스케일링
numerical_features = ['sequence_length', 'go_term_count', 'pubmed_count', 'globalMetricValue','ppi_degree', 'ppi_avg_score', 'pdb_count', 'gene_disease_count']
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(df_feat[numerical_features])

# 범주형 처리
categorical_features = ['organismScientificName']
df_feat[categorical_features] = df_feat[categorical_features].fillna('Unknown')
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_categorical = encoder.fit_transform(df_feat[categorical_features])

# 병합
feature_matrix = np.hstack([scaled_numerical, encoded_categorical])
df_feat['protein_index'] = df_feat['protein1'].map(protein_to_idx)

# 단백질별 평균 피처 생성
node_features = np.zeros((len(protein_to_idx), feature_matrix.shape[1]))
counts = df_feat['protein_index'].value_counts().to_dict()

for idx, row in df_feat.iterrows():
    p_idx = row['protein_index']
    if pd.notnull(p_idx):
        node_features[int(p_idx)] += feature_matrix[idx]

for prot, idx in protein_to_idx.items():
    count = counts.get(idx, 1)
    node_features[idx] /= count

x = torch.tensor(node_features, dtype=torch.float)

# 라벨 생성 (protein1에 포함되면 1, 아니면 0)
labeled_proteins = set(df['protein1'].dropna())
y = torch.tensor([1 if prot in labeled_proteins else 0 for prot in all_proteins], dtype=torch.long)

# PyG 데이터 객체
data = Data(x=x, edge_index=edge_index, y=y)
data

<ipython-input-6-1afbdc702a0d>:10: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DP/bio_datas/train_data.csv")


Data(x=[13378, 10], edge_index=[2, 340156], y=[13378])

# 2번째 GCN 피처 설정 및 학습

In [ ]:
import pandas as pd
import numpy as np
import torch
import networkx as nx
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
cols_to_drop = [
    'PubMed_IDs', 'entryId', 'sequenceChecksum', 'sequenceVersionDate',
    'uniprotDescription', 'organismScientificName', 'globalMetricValue',
    'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'modelCreatedDate',
    'latestVersion', 'allVersions', '_version_', 'proteinShortNames',
    'uniprotAccession_unchar', 'entry_name', 'protein_name', 'organism',
    'protein_existence', 'sequence_version', 'UniProt_ID', 'uniprotId',
    'gene_x', 'gene_y', 'organismScientificNameT', 'tax_id', 'taxId',
    'organismCommonNames', 'isAMdata', 'Protein_ID_Formatted',
    'isReviewed', 'isReferenceProteome', 'geneSynonyms'
]

df_main = df.drop(columns=cols_to_drop, errors='ignore')
print(df_main.shape)


(340156, 12)


In [ ]:
# 매핑 테이블 생성
id_map = df_main[['uniprotAccession', 'protein1']].dropna().drop_duplicates()
protein1_to_uniprot = dict(zip(id_map['protein1'], id_map['uniprotAccession']))
# protein2 매핑 추가
id_map_2 = df_main[['uniprotAccession', 'protein2']].dropna().drop_duplicates()
protein2_to_uniprot = dict(zip(id_map_2['protein2'], id_map_2['uniprotAccession']))

# 두 개 합치기
combined_mapping = {**protein1_to_uniprot, **protein2_to_uniprot}


In [ ]:
import torch
from torch_geometric.data import Data
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 노드 정의
proteins = pd.unique(df_main['uniprotAccession'])
protein_to_idx = {prot: idx for idx, prot in enumerate(proteins)}

# 엣지 정의
edges = df_main[['protein1', 'protein2']].dropna()
edges['p1_mapped'] = edges['protein1'].map(combined_mapping)
edges['p2_mapped'] = edges['protein2'].map(combined_mapping)
edges_valid = edges.dropna(subset=['p1_mapped', 'p2_mapped'])

edge_index = torch.tensor([
    [protein_to_idx[p1], protein_to_idx[p2]]
    for p1, p2 in edges_valid[['p1_mapped', 'p2_mapped']].values
], dtype=torch.long).t()

# 특성 정의 (GO_Terms, sequence 길이 등)
df_nodes = df_main.drop_duplicates('uniprotAccession')[['uniprotAccession', 'GO_Terms', 'sequence', 'PDB_IDs']]
df_nodes = df_nodes.set_index('uniprotAccession').loc[proteins]

vectorizer = TfidfVectorizer()
go_features = vectorizer.fit_transform(df_nodes['GO_Terms'].fillna('')).toarray()

has_structure = (~df_nodes['PDB_IDs'].isna()).astype(int).values.reshape(-1, 1)

# 6. 단백질 서열 → ESM 임베딩
data_sequences = [(idx, seq) for idx, seq in df_nodes['sequence'].items()]

In [ ]:
!pip install fair-esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import esm

# ✅ 사전학습된 ESM 모델과 알파벳 불러오기
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()

# ✅ 배치 컨버터 생성 (서열 데이터를 토큰으로 바꿔주는 도구)
batch_converter = alphabet.get_batch_converter()


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [ ]:
# 2. 모델을 GPU로 이동
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# 3. 단백질 서열 준비
data_sequences = [(idx, seq) for idx, seq in df_nodes['sequence'].items()]  # df_nodes는 uniprot 기준


In [ ]:
batch_size = 16
embeddings_list = []

for i in range(0, len(data_sequences), batch_size):
    batch_seqs = data_sequences[i:i+batch_size]
    batch_labels, batch_strs, batch_tokens = batch_converter(batch_seqs)

    with torch.no_grad():
        batch_tokens = batch_tokens.to(device)
        results = model(batch_tokens, repr_layers=[6], return_contacts=False)
        emb = results["representations"][6][:, 0, :]  # CLS token
        embeddings_list.append(emb.cpu())

embeddings = torch.cat(embeddings_list, dim=0)


In [ ]:
# 타겟 질병 설정 예시 (이중 하나를 타겟으로 설정)
target_disease = 'C0035126'
labels = torch.tensor([
    1 if target_disease in df_main[df_main['uniprotAccession'] == prot]['Disease ID'].values else 0
    for prot in proteins
], dtype=torch.long)

data = Data(x=node_features, edge_index=edge_index, y=labels)
print(data)

Data(x=[13378, 10], edge_index=[2, 312280], y=[29])


In [ ]:
print(data)
print(data.num_nodes, data.num_edges, data.num_node_features)

Data(x=[13378, 10], edge_index=[2, 312280], y=[29])
13378 312280 10


In [ ]:
print(f"data.x type: {type(data.x)}")  # Tensor여야 함
print(f"data.edge_index type: {type(data.edge_index)}")  # Tensor여야 함
print(f"data.x shape: {data.x.shape}")

data.x type: <class 'torch.Tensor'>
data.edge_index type: <class 'torch.Tensor'>
data.x shape: torch.Size([13378, 10])


# GCN 모델 정의
모델 학습을 통해 예측

In [ ]:
# 질병 단백질 ID 목록 추출
# labeled_proteins = set(df['uniprotAccession'].dropna().unique())

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split

# ✅ GCN 모델 (multi-label)
class GCN_MultiLabel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

proteins = list(all_proteins)
# ✅ 라벨 생성 (multi-label): 단백질 → 질병 매핑 (0/1)
disease_ids = sorted(df_main['Disease ID'].dropna().unique().tolist())
disease_to_idx = {d: i for i, d in enumerate(disease_ids)}

y_multi = torch.zeros((len(proteins), len(disease_ids)))
for i, prot in enumerate(proteins):
    related_diseases = df_main[df_main['uniprotAccession'] == prot]['Disease ID'].dropna().unique()
    for d in related_diseases:
        if d in disease_to_idx:
            y_multi[i][disease_to_idx[d]] = 1

# 1. 노드마다 질병이 있는지 확인 (y_multi는 [num_nodes, num_diseases])
has_disease = (y_multi.sum(dim=1) > 0)  # shape: [num_nodes]

# 2. 이진 라벨로 변환 → stratify 기준
labels_for_split = has_disease.int().numpy()

# 3. stratify 기반으로 인덱스 나누기
idx = np.arange(len(labels_for_split))
train_idx, test_idx = train_test_split(
    idx,
    test_size=0.2,
    stratify=labels_for_split,  # ⚠️ 여기가 포인트!
    random_state=42
)

# 4. PyTorch 마스크 생성
train_mask = torch.zeros(len(labels_for_split), dtype=torch.bool)
test_mask = torch.zeros(len(labels_for_split), dtype=torch.bool)
train_mask[train_idx] = True
test_mask[test_idx] = True

data.train_mask = train_mask
data.test_mask = test_mask

positive_counts = y_multi.sum(dim=0)
total_counts = y_multi.size(0)
pos_weight = (total_counts - positive_counts) / (positive_counts + 1e-5)
pos_weight = pos_weight.to(dtype=torch.float32)

# 하이퍼파라미터
in_channels = data.num_node_features
hidden_channels = 64
out_channels = y_multi.shape[1]

model = GCN_MultiLabel(in_channels, hidden_channels, out_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # 클래스 불균형 대응

# 학습 루프
for epoch in range(1, 101):
    model.train()
    optimizer.zero_grad()

    logits = model(data)
    loss = loss_fn(logits[data.train_mask], y_multi[data.train_mask])
    loss.backward()
    optimizer.step()

     # ✅ 평가
    model.eval()
    with torch.no_grad():
        probs = torch.sigmoid(logits)  # 직접 sigmoid 적용
        pred = (probs[data.test_mask] > 0.5).int()
        true = y_multi[data.test_mask].int()

        acc = (pred == true).float().mean().item()
        f1 = f1_score(true.cpu(), pred.cpu(), average='micro', zero_division=0)
        p = precision_score(true.cpu(), pred.cpu(), average='micro', zero_division=0)
        r = recall_score(true.cpu(), pred.cpu(), average='micro', zero_division=0)

        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f} | P: {p:.4f} | R: {r:.4f}")

Epoch 001 | Loss: 0.6932 | Acc: 0.5053 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 002 | Loss: 0.6544 | Acc: 0.7163 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 003 | Loss: 0.6155 | Acc: 0.8691 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 004 | Loss: 0.5730 | Acc: 0.9612 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 005 | Loss: 0.5260 | Acc: 0.9924 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 006 | Loss: 0.4749 | Acc: 0.9987 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 007 | Loss: 0.4209 | Acc: 1.0000 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 008 | Loss: 0.3657 | Acc: 1.0000 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 009 | Loss: 0.3110 | Acc: 1.0000 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 010 | Loss: 0.2587 | Acc: 1.0000 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 011 | Loss: 0.2105 | Acc: 1.0000 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 012 | Loss: 0.1678 | Acc: 1.0000 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 013 | Loss: 0.1313 | Acc: 1.0000 | F1: 0.0000 | P: 0.0000 | R: 0.0000
Epoch 014 | 

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score

# 1️⃣ protein index 정의
proteins = list(set(df_main['uniprotAccession'].dropna()))
protein_to_idx = {prot: i for i, prot in enumerate(proteins)}

# 2️⃣ edge_index 생성
edges = df_main[['protein1', 'protein2']].dropna().copy()
combined_mapping = {**dict(zip(df_main['protein1'], df_main['uniprotAccession'])),
                    **dict(zip(df_main['protein2'], df_main['uniprotAccession']))}
edges['p1'] = edges['protein1'].map(combined_mapping)
edges['p2'] = edges['protein2'].map(combined_mapping)
edges = edges.dropna(subset=['p1', 'p2'])

edge_index = torch.tensor([
    [protein_to_idx[p1], protein_to_idx[p2]]
    for p1, p2 in edges[['p1', 'p2']].values
    if p1 in protein_to_idx and p2 in protein_to_idx
], dtype=torch.long).t()

# 3️⃣ node feature: sequence length + structure 여부
df_nodes = df_main.drop_duplicates('uniprotAccession')[['uniprotAccession', 'sequence', 'PDB_IDs']]
df_nodes = df_nodes.set_index('uniprotAccession').loc[proteins]
seq_len = df_nodes['sequence'].fillna('').apply(len).values.reshape(-1, 1)
has_structure = (~df_nodes['PDB_IDs'].isna()).astype(int).values.reshape(-1, 1)
features = np.concatenate([seq_len, has_structure], axis=1)
features = StandardScaler().fit_transform(features)
x = torch.tensor(features, dtype=torch.float)

# 4️⃣ 라벨 생성 (multi-label)
diseases = sorted(df_main['Disease ID'].dropna().unique().tolist())
disease_to_idx = {d: i for i, d in enumerate(diseases)}

y_multi = torch.zeros((len(proteins), len(diseases)))
for i, prot in enumerate(proteins):
    rel_dis = df_main[df_main['uniprotAccession'] == prot]['Disease ID'].dropna().unique()
    for d in rel_dis:
        if d in disease_to_idx:
            y_multi[i][disease_to_idx[d]] = 1

# 5️⃣ 희귀 질병 제거
min_count = 5
keep_cols = (y_multi.sum(dim=0) >= min_count).nonzero().flatten()
y_multi = y_multi[:, keep_cols]

# 6️⃣ train/test split (stratify 사용)
has_disease = (y_multi.sum(dim=1) > 0)
idx = np.arange(len(proteins))
train_idx, test_idx = train_test_split(
    idx, test_size=0.2, stratify=has_disease.int().numpy(), random_state=42
)
train_mask = torch.zeros(len(proteins), dtype=torch.bool)
test_mask = torch.zeros(len(proteins), dtype=torch.bool)
train_mask[train_idx] = True
test_mask[test_idx] = True

# 7️⃣ class imbalance 보정
pos = y_multi[train_mask].sum(0)
neg = (train_mask.sum() - pos)
pos_weight = (neg / (pos + 1e-6)).clamp(min=1.0)

# 8️⃣ GCN 모델
class GCN_MultiLabel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

model = GCN_MultiLabel(x.shape[1], 64, y_multi.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

data = Data(x=x, edge_index=edge_index)
data.train_mask = train_mask
data.test_mask = test_mask

# 9️⃣ 학습
for epoch in range(1, 101):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out[train_mask], y_multi[train_mask])
    loss.backward()
    optimizer.step()

    # 평가
    model.eval()
    with torch.no_grad():
        logits = model(data)
        probs = torch.sigmoid(logits[test_mask])
        preds = (probs > 0.5).int()
        true = y_multi[test_mask].int()

        acc = (preds == true).float().mean().item()

        # ⛳️ 수정된 부분
        true_np = true.cpu().numpy()
        preds_np = preds.cpu().numpy()

        f1 = f1_score(true_np, preds_np, average='micro', zero_division=0)
        precision = precision_score(true_np, preds_np, average='micro', zero_division=0)
        recall = recall_score(true_np, preds_np, average='micro', zero_division=0)

        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f} | P: {precision:.4f} | R: {recall:.4f}")


ValueError: unknown is not supported

In [ ]:
print("✅ Test set에 실제 질병 1 비율:", y_multi[data.test_mask].sum().item() / y_multi[data.test_mask].numel())

✅ Test set에 실제 질병 1 비율: 0.0


In [ ]:
def predict_diseases_by_protein(protein_id, threshold=0.1, top_k=None):
    model.eval()

    # 단백질 인덱스 가져오기
    if protein_id not in protein_to_idx:
        raise ValueError(f"Unknown protein ID: {protein_id}")
    idx = protein_to_idx[protein_id]

    # 예측 수행
    with torch.no_grad():
        out = model(data)
        probs = out[idx].cpu().numpy()  # shape: [num_diseases]

    # 질병 리스트 + 확률
    disease_prob_pairs = [
        (disease_ids[i], float(prob))
        for i, prob in enumerate(probs)
        if prob > threshold  # threshold 이상인 것만
    ]

    # 확률 내림차순 정렬
    disease_prob_pairs.sort(key=lambda x: x[1], reverse=True)

    # 상위 N개만 (선택)
    if top_k:
        disease_prob_pairs = disease_prob_pairs[:top_k]

    return disease_prob_pairs


In [ ]:
protein = "Q9HAZ2"
predicted = predict_diseases_by_protein(protein, threshold=0.1, top_k=5)

print(f"🔍 단백질 {protein} 관련 질병 예측:")
for disease, prob in predicted:
    print(f"🦠 {disease}: {prob:.4f}")

🔍 단백질 Q9HAZ2 관련 질병 예측:


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# 평가
model.eval()
with torch.no_grad():
    pred = (out[data.test_mask] > 0.5).int().cpu()
    true = y_multi[data.test_mask].int().cpu()

    acc = (pred == true).float().mean().item()
    f1 = f1_score(true, pred, average='micro', zero_division=0)
    p = precision_score(true, pred, average='micro', zero_division=0)
    r = recall_score(true, pred, average='micro', zero_division=0)

    print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f} | P: {p:.4f} | R: {r:.4f}")

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# flatten 후 평가
f1 = f1_score(true.cpu().numpy().flatten(), pred.cpu().numpy().flatten())
precision = precision_score(true.cpu().numpy().flatten(), pred.cpu().numpy().flatten())
recall = recall_score(true.cpu().numpy().flatten(), pred.cpu().numpy().flatten())

print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f} | P: {precision:.4f} | R: {recall:.4f}")

Epoch 100 | Loss: 0.0001 | Acc: 1.0000 | F1: 0.0000 | P: 0.0000 | R: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print("1의 비율 (True):", true.sum().item() / true.numel())
print("1의 비율 (Pred):", pred.sum().item() / pred.numel())

1의 비율 (True): 0.0
1의 비율 (Pred): 0.0


# GAT 모델 정의 및 학습

In [ ]:
# 표준 라이브러리
import numpy as np
import pandas as pd
import networkx as nx

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

# PyTorch Geometric
from torch_geometric.data import Data
from torch_geometric.nn import GATConv


class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(GAT, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        return x


In [ ]:
# Google Colab에 맞게 경로 설정 및 파일 업로드 지원 포함
import numpy as np
import pandas as pd
import networkx as nx

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GATConv

# Colab에서 파일 업로드 지원
from google.colab import files
uploaded = files.upload()  # 파일 업로드
csv_path = list(uploaded.keys())[0]  # 첫 번째 업로드된 파일 사용

class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(GAT, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        return x


class ProteinDiseasePredictor:
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
        self.data = None
        self.label_encoder = LabelEncoder()
        self.protein_to_idx = {}
        self.idx_to_protein = {}
        self.idx_to_disease = {}
        self.disease_labels = None

    def preprocess_data(self):
        df = self.df.copy()
        all_proteins = pd.unique(df[['protein1', 'protein2']].values.ravel())
        self.protein_to_idx = {protein: idx for idx, protein in enumerate(all_proteins)}
        self.idx_to_protein = {idx: protein for protein, idx in self.protein_to_idx.items()}

        edges = df[['protein1', 'protein2']].dropna().values
        edge_index = torch.tensor([[self.protein_to_idx[a], self.protein_to_idx[b]] for a, b in edges], dtype=torch.long).T

        df['sequence'] = df['sequence'].fillna('')
        df['GO_Terms'] = df['GO_Terms'].fillna('')
        df['PubMed_IDs'] = df['PubMed_IDs'].fillna('') if 'PubMed_IDs' in df.columns else ''
        df['globalMetricValue'] = df['globalMetricValue'].fillna(0)
        df['organismScientificName'] = df.get('organismScientificName', pd.Series(['Unknown'] * len(df))).fillna('Unknown')
        df['sequence_length'] = df['sequence'].apply(len)
        df['go_term_count'] = df['GO_Terms'].apply(lambda x: len(x.split(';')) if x else 0)
        df['pubmed_count'] = df['PubMed_IDs'].apply(lambda x: len(str(x).split(';')) if pd.notnull(x) else 0) if 'PubMed_IDs' in df.columns else 0

        ppi_edges = df[['protein1', 'protein2', 'combined_score']].dropna()
        ppi_graph = nx.from_pandas_edgelist(ppi_edges, source='protein1', target='protein2', edge_attr='combined_score')
        ppi_degree = dict(ppi_graph.degree())
        ppi_avg_score = {node: np.mean([d['combined_score'] for _, _, d in ppi_graph.edges(node, data=True)]) for node in ppi_graph.nodes()}
        df['ppi_degree'] = df['protein1'].map(ppi_degree).fillna(0)
        df['ppi_avg_score'] = df['protein1'].map(ppi_avg_score).fillna(0)
        df['pdb_count'] = df['PDB_IDs'].fillna('').apply(lambda x: len(x.split(';')) if x else 0)
        gene_disease_count = df.groupby('Gene ID')['Disease ID'].nunique()
        df['gene_disease_count'] = df['Gene ID'].map(gene_disease_count).fillna(0)

        numerical_features = ['sequence_length', 'go_term_count', 'pubmed_count', 'globalMetricValue',
                              'ppi_degree', 'ppi_avg_score', 'pdb_count', 'gene_disease_count']
        scaler = StandardScaler()
        scaled_numerical = scaler.fit_transform(df[numerical_features])

        categorical_features = ['organismScientificName']
        df[categorical_features] = df[categorical_features].fillna('Unknown')
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded_categorical = encoder.fit_transform(df[categorical_features])

        feature_matrix = np.hstack([scaled_numerical, encoded_categorical])

        df['protein_index'] = df['protein1'].map(self.protein_to_idx)
        node_features = np.zeros((len(self.protein_to_idx), feature_matrix.shape[1]))
        counts = df['protein_index'].value_counts().to_dict()
        for idx, row in df.iterrows():
            p_idx = row['protein_index']
            if pd.notnull(p_idx):
                node_features[int(p_idx)] += feature_matrix[idx]
        for prot, idx in self.protein_to_idx.items():
            count = counts.get(idx, 1)
            node_features[idx] /= count

        x = torch.tensor(node_features, dtype=torch.float)

        df = df.dropna(subset=['Disease ID'])
        df = df.groupby('protein1').first().reset_index()
        y_raw = df[['protein1', 'Disease ID']]
        self.label_encoder.fit(y_raw['Disease ID'])
        self.idx_to_disease = {i: v for i, v in enumerate(self.label_encoder.classes_)}
        y_dict = dict(zip(y_raw['protein1'], self.label_encoder.transform(y_raw['Disease ID'])))
        y = torch.tensor([y_dict.get(prot, -1) for prot in self.protein_to_idx.keys()])

        mask = y != -1
        self.data = Data(x=x, edge_index=edge_index, y=y, train_mask=mask, test_mask=mask)

    def train(self, hidden_channels=64, epochs=100, lr=0.01):
        data = self.data
        train_idx, test_idx = train_test_split(torch.where(data.train_mask)[0], test_size=0.2, stratify=data.y[data.train_mask], random_state=42)

        data.train_mask[:] = False
        data.test_mask[:] = False
        data.train_mask[train_idx] = True
        data.test_mask[test_idx] = True

        in_channels = data.num_node_features
        out_channels = len(self.label_encoder.classes_)
        model = GAT(in_channels, hidden_channels, out_channels)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.CrossEntropyLoss()

        for epoch in range(1, epochs + 1):
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            pred = out.argmax(dim=1)
            correct = pred[data.test_mask] == data.y[data.test_mask]
            acc = int(correct.sum()) / int(data.test_mask.sum())

            print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {acc:.4f}")

        self.model = model

    def predict(self):
        self.model.eval()
        out = self.model(self.data)
        pred = out.argmax(dim=1)
        protein_predictions = {
            self.idx_to_protein[idx]: self.idx_to_disease[pred[idx].item()] for idx in range(len(pred)) if self.data.train_mask[idx]
        }
        predictions_df = pd.DataFrame(list(protein_predictions.items()), columns=['Protein', 'Predicted_Disease_ID'])
        print(predictions_df.head())
        return predictions_df

# 실행 예시
predictor = ProteinDiseasePredictor(csv_path)
predictor.preprocess_data()
predictor.train()
predictor.predict()


ModuleNotFoundError: No module named 'torch_geometric'

## ipynb로 어느정도 1차 목표까지 했으면 .py로 변환하고, class화도 하고 유지보수 하기 쉽게 정리해줘야함.

# 단백질 분포도에 대한 시각화

In [ ]:
# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# combined_score에 대한 분포 시각화
sns.displot(ppi_merged["combined_score"], kde=True)  # kde=True는 밀도 추정을 추가합니다.
plt.show()


# 시각화
단백질 간에 연관성 스코어를 가지고 시각화를 한다. \
그 단백질이 어느 질병과 연관이 있으며 그 단백질은 어떤 단백질하고 연관이 있는지 파악할 수 있다.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# 예시: 특정 질병만 추출
target_disease = 'C0036095'
sub_df = ppi_merged[ppi_merged['Disease ID'] == target_disease]

# 그래프 생성
G = nx.Graph()

# edge 추가 (protein1 - protein2)
G = nx.Graph()
for _, row in sub_df[sub_df['combined_score'] > 500].iterrows():
    G.add_edge(row['protein1'], row['protein2'], weight=row['combined_score'])

# 시각화
plt.figure(figsize=(10,10))
nx.draw_networkx(G, with_labels=False, node_size=30, alpha=0.7)
plt.title(f"PPI Network for Disease: {target_disease}")
plt.show()
